# Análise Exploratória de Dados

## Preparação

In [4]:
import warnings

import polars as pl
from tabulate import tabulate

from df_utils import (
    get_list_column_max_len,
    normalize_schemas,
    one_hot_encode_list_column,
)
from geo_location import extract_polygons_from_folder, mark_points_in_polygons

warnings.filterwarnings('ignore', category=pl.exceptions.MapWithoutReturnDtypeWarning)

## Leitura de dados

Os dados se encontram no formato NDJSON, também conhecido como JSONLines,
em que há um objeto JSON por linha no arquivo.

In [5]:
neighborhoods = ["cidade_baixa", "centro_historico", "menino_deus", "sarandi"]

neighborhoods_df_map = {
    neighborhood: pl.read_ndjson(f"data/{neighborhood}_listings.json")
    for neighborhood in neighborhoods
}

## Entendimento dos dados e pré-processamento

Gostaríamos de concatenar os dados das diferentes fontes, para facilitar a análise.

In [6]:
try:
    pl.concat(neighborhoods_df_map.values())
except pl.exceptions.SchemaError:
    print("Os dados não têm esquema homogêneo.")

Os dados não têm esquema homogêneo.


A concatenação direta não é possível, pois a estrutura dos dados não é homogênea.

In [7]:
for name, df in neighborhoods_df_map.items():
    print(name)
    print("=" * len(name))
    print(df.schema)
    print()

cidade_baixa
Schema([('listing', Struct({'stamps': List(String), 'legacyId': String, 'createdAt': String, 'usableAreas': List(Int64), 'title': String, 'contractType': String, 'sourceId': String, 'id': String, 'portal': String, 'acceptExchange': Boolean, 'advertiserContact': Struct({'chat': String, 'phones': List(String)}), 'buildings': Int64, 'usageTypes': List(String), 'unitTypes': List(String), 'updatedAt': String, 'displayAddressType': String, 'listingsCount': Int64, 'constructionStatus': String, 'unitFloor': Int64, 'suites': List(Int64), 'unitSubTypes': List(String), 'unitsOnTheFloor': Int64, 'externalId': String, 'bedrooms': List(Int64), 'amenities': List(String), 'floors': List(Int64), 'description': String, 'pricingInfos': List(Struct({'businessType': String, 'yearlyIptu': Int64, 'price': Int64, 'monthlyCondoFee': Int64, 'rentalInfo': Struct({'period': String, 'warranties': List(Null)})})), 'nonActivationReason': String, 'parkingSpaces': List(Int64), 'listingType': String, 'bath

A maior diferença parece estar na estrutura aninhada `rentalInfo`,
que não é presente nos dados vindos do bairro Sarandi.

In [8]:
dfs = list(neighborhoods_df_map.values())
normalized_dfs = normalize_schemas(dfs)
df = pl.concat(normalized_dfs)

In [9]:
df.head(2)

listing,account,medias,accountLink,link
struct[43],struct[9],list[struct[3]],struct[4],struct[4]
"{[],""26349587"",""2019-11-05T13:09:45.557+00:00"",[46],""Apartamento de 46 metros quadrados no bairro Cidade Baixa com 1 quarto"",""OWNER"",""a5adbade-589d-3c61-97b7-e4014f21cf3d"",""2463567709"",""GRUPOZAP"",true,{"""",[""5130130100""]},0,[""RESIDENTIAL""],[""APARTMENT""],""2024-12-17T11:12:11.169+00:00"",""STREET"",null,""ConstructionStatus_NONE"",0,[],[],4,""ZAP1593841"",[1],[""DISABLED_ACCESS"", ""INTEGRATED_ENVIRONMENTS"", … ""PARTY_HALL""],[8],""Apartamento em excelente estado, iluminado e silencioso, mobiliado com móveis embutidos , conceito aberto, cozinha e sala integrados, gás central, aquecimento em todas as torneiras, fica ar condicionado do quarto e todos os móveis embutidos quarto, banheiro, sala, cozinha e homeoffice."",[{""SALE"",600,290000,300,{null,[],null}}],""NonActivationReason_NONE"",[],""USED"",[1],"""",true,false,""UNIT"",""ACTIVE"",{""Porto Alegre"",""Cidade Baixa"",""Rua Joaquim Nabuco"",null,{""GOOGLE"",-30.041,-51.219,140,null,null},""RS""},""PREMIUM"","""",""dfadbe19-2554-5480-56eb-0780d0ee4019"",[48],[0]}","{""dfadbe19-2554-5480-56eb-0780d0ee4019"",""BÁRBARA EINSFELD DE BORBA"",null,"""",true,345436,3136761,""2018-05-11T05:14:21Z"",""""}","[{""7cb4b1098bf75603a7f2561456bc2883"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/7cb4b1098bf75603a7f2561456bc2883.webp"",""IMAGE""}, {""3ad45448360d515b4bbb3d833537b5e4"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/3ad45448360d515b4bbb3d833537b5e4.webp"",""IMAGE""}, … {""3c74183638571539a2c2f9f568a72d7a"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/3c74183638571539a2c2f9f568a72d7a.webp"",""IMAGE""}]","{""BÁRBARA EINSFELD DE BORBA"",""/imobiliaria/345436/"","""",{}}","{""Apartamento com 1 Quarto à venda, 46m²"",""/imovel/venda-apartamento-1-quarto-mobiliado-cidade-baixa-porto-alegre-46m2-id-2463567709/"","""",{""Porto Alegre"",""Cidade Baixa"","""",""Rua Joaquim Nabuco"","""",""""}}"
"{[],"""",""2024-12-01T21:22:02.001+00:00"",[56],""Compre apartamento silencioso,55,10m2 de área útil,2 quartos, piso parquet,living 2 ambientes,banhei"",""REAL_ESTATE"",""40017671-36da-3cb2-98e7-116dd4abcba1"",""2760694515"",""GRUPOZAP"",false,{"""",[""5132084035"", ""51999671489""]},0,[""RESIDENTIAL""],[""APARTMENT""],""2024-12-26T12:00:41.422+00:00"",""ALL"",7,""ConstructionStatus_NONE"",0,[0],[],0,""VR527903"",[2],[],[],""Compre apartamento silencioso,55,10m2 de área útil,2 quartos, piso parquet,living 2 ambientes,banheiro social ,cozinha e área de serviço. O condomínio fica localizado em Rua José do Patrocínio no bairro Cidade Baixa em Porto Alegre. Está bem situado, próximo a pontos de interesse de Cidade Baixa, tais como Azambuja, Praça General Daltro Filho, Faculdade de Direito da Funda. Escola Sup. do Ministério Público, Praça Salvador Allende, e Policlínica Militar de Porto Alegre.Aceitamos Fiinanciamento Bancário e Fgts.Marque hoje mesmo uma visita com um de nossos Consultores Imobiliários caddstrados! ]]>"",[{""SALE"",0,275000,290,{null,[],null}}],""NonActivationReason_NONE"",[],""USED"",[1],""51999671489"",true,false,""UNIT"",""ACTIVE"",{""Porto Alegre"",""Cidade Baixa"",""Rua José do Patrocínio"",""357"",{""GOOGLE"",null,null,null,-30.038562,-51.224749},""RS""},""PREMIUM"",""13887"",""90879777-b9cc-65b9-28c8-c86f00947af9"",[66],[]}","{""90879777-b9cc-65b9-28c8-c86f00947af9"",""Sperinde Vendas"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/fb5adab24465e4ba59ba7d6e4ac1de9e.webp"",""00411-J-RS"",true,33518,2594637,""2018-03-27T18:49:45Z"",""diamond""}","[{""3d8f346437754cf315cc942816497178"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/3d8f346437754cf315cc942816497178.webp"",""IMAGE""}, {""d309f06d26092646e716b47e93936c7d"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/d3

A maior parte das informações que consideramos importantes para nossa análise estão
na coluna `listing`, que tem uma estrutura aninhada complexa.
Aplainamos a estrutura para simplificar a análise, e ainda mantemos o link do anúncio,
presente na coluna `link`, e informações do anunciante (`account`)
para facilitar a consulta para análises mais aprofundadas.

In [10]:
df = df.select([pl.col("listing").struct.unnest(), pl.col("account"), pl.col("link")])
df = df.rename({"id": "listingId"})
df = df.select([pl.all().exclude("account"), pl.col("account").struct.unnest()])
df = df.rename(
    {
        "id": "accountId",
        "name": "accountName",
        "createdDate": "accountCreatedDate",
        "tier": "accountTier",
    }
)
df = df.select(
    [pl.all().exclude("link"), pl.col("link").struct.field("href").alias("link_href")]
)

In [11]:
df.head(2)

stamps,legacyId,createdAt,usableAreas,title,contractType,sourceId,listingId,portal,acceptExchange,advertiserContact,buildings,usageTypes,unitTypes,updatedAt,displayAddressType,listingsCount,constructionStatus,unitFloor,suites,unitSubTypes,unitsOnTheFloor,externalId,bedrooms,amenities,floors,description,pricingInfos,nonActivationReason,parkingSpaces,listingType,bathrooms,whatsappNumber,showPrice,resale,propertyType,status,address,publicationType,providerId,advertiserId,totalAreas,capacityLimit,accountId,accountName,logoUrl,licenseNumber,showAddress,legacyVivarealId,legacyZapId,accountCreatedDate,accountTier,link_href
list[str],str,str,list[i64],str,str,str,str,str,bool,struct[2],i64,list[str],list[str],str,str,i64,str,i64,list[i64],list[str],i64,str,list[i64],list[str],list[i64],str,list[struct[5]],str,list[i64],str,list[i64],str,bool,bool,str,str,struct[6],str,str,str,list[i64],list[i64],str,str,str,str,bool,i64,i64,str,str,str
[],"""26349587""","""2019-11-05T13:09:45.557+00:00""",[46],"""Apartamento de 46 metros quadr…","""OWNER""","""a5adbade-589d-3c61-97b7-e4014f…","""2463567709""","""GRUPOZAP""",true,"{"""",[""5130130100""]}",0,"[""RESIDENTIAL""]","[""APARTMENT""]","""2024-12-17T11:12:11.169+00:00""","""STREET""",null,"""ConstructionStatus_NONE""",0,[],[],4,"""ZAP1593841""",[1],"[""DISABLED_ACCESS"", ""INTEGRATED_ENVIRONMENTS"", … ""PARTY_HALL""]",[8],"""Apartamento em excelente estad…","[{""SALE"",600,290000,300,{null,[],null}}]","""NonActivationReason_NONE""",[],"""USED""",[1],"""""",true,false,"""UNIT""","""ACTIVE""","{""Porto Alegre"",""Cidade Baixa"",""Rua Joaquim Nabuco"",null,{""GOOGLE"",-30.041,-51.219,140,null,null},""RS""}","""PREMIUM""","""""","""dfadbe19-2554-5480-56eb-0780d0…",[48],[0],"""dfadbe19-2554-5480-56eb-0780d0…","""BÁRBARA EINSFELD DE BORBA""",null,"""""",true,345436,3136761,"""2018-05-11T05:14:21Z""","""""","""/imovel/venda-apartamento-1-qu…"
[],"""""","""2024-12-01T21:22:02.001+00:00""",[56],"""Compre apartamento silencioso,…","""REAL_ESTATE""","""40017671-36da-3cb2-98e7-116dd4…","""2760694515""","""GRUPOZAP""",false,"{"""",[""5132084035"", ""51999671489""]}",0,"[""RESIDENTIAL""]","[""APARTMENT""]","""2024-12-26T12:00:41.422+00:00""","""ALL""",7,"""ConstructionStatus_NONE""",0,[0],[],0,"""VR527903""",[2],[],[],"""Compre apartamento silencioso,…","[{""SALE"",0,275000,290,{null,[],null}}]","""NonActivationReason_NONE""",[],"""USED""",[1],"""51999671489""",true,false,"""UNIT""","""ACTIVE""","{""Porto Alegre"",""Cidade Baixa"",""Rua José do Patrocínio"",""357"",{""GOOGLE"",null,null,null,-30.038562,-51.224749},""RS""}","""PREMIUM""","""13887""","""90879777-b9cc-65b9-28c8-c86f00…",[66],[],"""90879777-b9cc-65b9-28c8-c86f00…","""Sperinde Vendas""","""https://resizedimgs.zapimoveis…","""00411-J-RS""",true,33518,2594637,"""2018-03-27T18:49:45Z""","""diamond""","""/imovel/venda-apartamento-2-qu…"


Qual o tamanho da massa de dados?

In [12]:
df.height

17341

Obtivemos 17.341 amostras.

Quantas amostras de cada bairro?

In [13]:
df = df.unnest("address")

In [14]:
df.group_by("neighborhood").len().sort(by="len", descending=True)

neighborhood,len
str,u32
"""Centro Histórico""",6407
"""Menino Deus""",5215
"""Sarandi""",2615
"""Cidade Baixa""",2052
"""Santa Tereza""",268
…,…
"""Jardim Europa""",1
"""Agronomia""",1
"""Humaitá""",1


Nota-se que o processo de extração trouxe dados de outros bairros, além dos desejados,
mas em menor quantidade.

In [15]:
target_neighborhoods = ["Centro Histórico", "Cidade Baixa", "Menino Deus", "Sarandi"]

In [16]:
df = df.filter(pl.col("neighborhood").is_in(target_neighborhoods))

In [17]:
df.group_by("neighborhood").len().sort(by="len", descending=True)

neighborhood,len
str,u32
"""Centro Histórico""",6407
"""Menino Deus""",5215
"""Sarandi""",2615
"""Cidade Baixa""",2052


In [18]:
df.height

16289

Restam 16.289 amostras.

A massa de dados é relativamente pequena, totalizando 16.289 imóveis.

Existem dados duplicados?

In [19]:
df.filter(pl.col("listingId").is_duplicated()).height

1708

Sim, 1.708 amostras são duplicadas.

In [20]:
df = df.unique(subset="listingId")

In [21]:
df.height

15432

In [22]:
df.group_by("neighborhood").len().sort(by="len", descending=True)

neighborhood,len
str,u32
"""Centro Histórico""",6059
"""Menino Deus""",4815
"""Sarandi""",2551
"""Cidade Baixa""",2007


### Descarte de informações não importantes

Existe muita informação ruidosa nesses dados.
Escolhemos as seguintes colunas para descarte.

In [23]:
df.head(1)

stamps,legacyId,createdAt,usableAreas,title,contractType,sourceId,listingId,portal,acceptExchange,advertiserContact,buildings,usageTypes,unitTypes,updatedAt,displayAddressType,listingsCount,constructionStatus,unitFloor,suites,unitSubTypes,unitsOnTheFloor,externalId,bedrooms,amenities,floors,description,pricingInfos,nonActivationReason,parkingSpaces,listingType,bathrooms,whatsappNumber,showPrice,resale,propertyType,status,city,neighborhood,street,streetNumber,point,stateAcronym,publicationType,providerId,advertiserId,totalAreas,capacityLimit,accountId,accountName,logoUrl,licenseNumber,showAddress,legacyVivarealId,legacyZapId,accountCreatedDate,accountTier,link_href
list[str],str,str,list[i64],str,str,str,str,str,bool,struct[2],i64,list[str],list[str],str,str,i64,str,i64,list[i64],list[str],i64,str,list[i64],list[str],list[i64],str,list[struct[5]],str,list[i64],str,list[i64],str,bool,bool,str,str,str,str,str,str,struct[6],str,str,str,str,list[i64],list[i64],str,str,str,str,bool,i64,i64,str,str,str
[],"""""","""2024-11-04T20:12:31.282+00:00""","[37, 38]","""Apartamento em Cidade Baixa co…","""REAL_ESTATE""","""67d13ced-652c-31b1-ae45-a7176d…","""2754214663""","""GRUPOZAP""",false,"{"""",[""5192072370""]}",0,"[""RESIDENTIAL""]","[""APARTMENT""]","""2024-12-26T07:15:52.948+00:00""","""ALL""",7,"""ConstructionStatus_NONE""",0,[],[],0,"""39626""",[1],"[""INTERCOM"", ""KITCHEN""]",[],"""Este encantador apartamento de…","[{""SALE"",1500,200000,300,{null,[],null}}]","""NonActivationReason_NONE""",[0],"""USED""",[1],"""""",true,false,"""UNIT""","""ACTIVE""","""Porto Alegre""","""Cidade Baixa""","""Rua Luiz Afonso""","""558""","{""GOOGLE"",null,null,null,-30.041164,-51.223899}","""RS""","""PREMIUM""","""77942""","""8c109097-7891-1cd2-f172-ac8f5c…","[1, 43]",[],"""8c109097-7891-1cd2-f172-ac8f5c…","""Doorz Serviços Imobiliários""","""https://resizedimgs.zapimoveis…","""27212-J-RS""",false,814353,0,"""2023-09-06T13:48:21Z""","""diamond""","""/imovel/venda-apartamento-1-qu…"


In [24]:
COLUMNS_TO_DROP = [
    "advertiserId",
    "constructionStatus",
    "portal",
    "stamps",
    "advertiserContact",
    "whatsappNumber",
    "title",
    "nonActivationReason",
    "status",
    "legacyId",
    "externalId",
    "listingsCount",
    "createdAt",
    "updatedAt",
    "showPrice",
    "acceptExchange",
    "description",
    "sourceId",
    "providerId",
    "accountName",
    "accountCreatedDate",
    "accountTier",
    "licenseNumber",
    "logoUrl",
    "legacyVivarealId",
    "legacyZapId",
    "showAddress"
]

df = df.drop(COLUMNS_TO_DROP)

In [25]:
df.head(3)

usableAreas,contractType,listingId,buildings,usageTypes,unitTypes,displayAddressType,unitFloor,suites,unitSubTypes,unitsOnTheFloor,bedrooms,amenities,floors,pricingInfos,parkingSpaces,listingType,bathrooms,resale,propertyType,city,neighborhood,street,streetNumber,point,stateAcronym,publicationType,totalAreas,capacityLimit,accountId,link_href
list[i64],str,str,i64,list[str],list[str],str,i64,list[i64],list[str],i64,list[i64],list[str],list[i64],list[struct[5]],list[i64],str,list[i64],bool,str,str,str,str,str,struct[6],str,str,list[i64],list[i64],str,str
"[37, 38]","""REAL_ESTATE""","""2754214663""",0,"[""RESIDENTIAL""]","[""APARTMENT""]","""ALL""",0,[],[],0,[1],"[""INTERCOM"", ""KITCHEN""]",[],"[{""SALE"",1500,200000,300,{null,[],null}}]",[0],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Cidade Baixa""","""Rua Luiz Afonso""","""558""","{""GOOGLE"",null,null,null,-30.041164,-51.223899}","""RS""","""PREMIUM""","[1, 43]",[],"""8c109097-7891-1cd2-f172-ac8f5c…","""/imovel/venda-apartamento-1-qu…"
[131],"""REAL_ESTATE""","""2742418167""",0,"[""COMMERCIAL""]","[""OFFICE""]","""ALL""",0,[],[],0,[],[],[],"[{""SALE"",1500,657300,0,{null,[],null}}]",[],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Centro Histórico""","""Rua Uruguai""","""155""","{""GOOGLE"",null,null,null,-30.028111,-51.22904}","""RS""","""STANDARD""",[175],[],"""dc0e9379-0d98-2ada-1128-b4e009…","""/imovel/venda-conjunto-comerci…"
[34],"""REAL_ESTATE""","""2596966563""",0,"[""RESIDENTIAL""]","[""APARTMENT""]","""ALL""",4,[1],[],0,[1],"[""KITCHEN"", ""FURNISHED""]",[4],"[{""SALE"",350,135000,200,{null,[],null}}]",[],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Centro Histórico""","""Avenida João Pessoa""","""403""","{""GOOGLE"",null,null,null,-30.035665,-51.220879}","""RS""","""PREMIUM""",[41],[],"""8ce30dcf-d575-1b69-a46d-c41c78…","""/imovel/venda-apartamento-1-qu…"


### Atributos em forma de lista

Sabemos que `amenities` descrevem as amenidades presentes em um dado imóvel,
o que pode ser interessante transformar em atributos preditores para nossas regressões.

Entretanto, ainda precisamos entender outras colunas que tem listas como valores.
Várias delas não parecem fazer sentido em ter multiplicidade: em que contexto o número
de quartos (bedrooms) precisa ser uma lista? Número de banheiros? Área utilizável?
Uma unidade pode pertencer ao mesmo tempo a mais de um tipo (comercial, casa)?

Em primeiro lugar, tentamos entender quais listas de fato vêm a possuir mais de um
elemento.


In [26]:
LIST_COLUMNS = [
    "floors",
    "unitSubTypes",
    "suites",
    "unitTypes",
    "pricingInfos",
    "parkingSpaces",
    "totalAreas",
    "bathrooms",
    "bedrooms",
    "usableAreas",
    "usageTypes",
    "capacityLimit",
]

max_colname_width = max(len(name) for name in LIST_COLUMNS)
arrays_with_size_larger_than_one = []
for colname in LIST_COLUMNS:
    column_max_length = get_list_column_max_len(df, colname)
    print(f"{colname:<{max_colname_width}}: {column_max_length}")
    if column_max_length > 1:
        arrays_with_size_larger_than_one.append(colname)

floors       : 1
unitSubTypes : 2
suites       : 1
unitTypes    : 1
pricingInfos : 2
parkingSpaces: 2
totalAreas   : 2
bathrooms    : 2
bedrooms     : 2
usableAreas  : 2
usageTypes   : 2
capacityLimit: 1


A primeira coisa a se fazer, para facilitar o restante da nossa análise e pré-processamento,
é transformar aquelas colunas que são sempre listas de um elemento em colunas escalares.

In [27]:
single_element_list_columns = [
    "capacityLimit",
    "unitTypes",
    "floors",
    "suites",
]

df = df.with_columns(
    [pl.col(col).list.first().alias(col) for col in single_element_list_columns]
)

In [28]:
df.head(2)

usableAreas,contractType,listingId,buildings,usageTypes,unitTypes,displayAddressType,unitFloor,suites,unitSubTypes,unitsOnTheFloor,bedrooms,amenities,floors,pricingInfos,parkingSpaces,listingType,bathrooms,resale,propertyType,city,neighborhood,street,streetNumber,point,stateAcronym,publicationType,totalAreas,capacityLimit,accountId,link_href
list[i64],str,str,i64,list[str],str,str,i64,i64,list[str],i64,list[i64],list[str],i64,list[struct[5]],list[i64],str,list[i64],bool,str,str,str,str,str,struct[6],str,str,list[i64],i64,str,str
"[37, 38]","""REAL_ESTATE""","""2754214663""",0,"[""RESIDENTIAL""]","""APARTMENT""","""ALL""",0,null,[],0,[1],"[""INTERCOM"", ""KITCHEN""]",null,"[{""SALE"",1500,200000,300,{null,[],null}}]",[0],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Cidade Baixa""","""Rua Luiz Afonso""","""558""","{""GOOGLE"",null,null,null,-30.041164,-51.223899}","""RS""","""PREMIUM""","[1, 43]",null,"""8c109097-7891-1cd2-f172-ac8f5c…","""/imovel/venda-apartamento-1-qu…"
[131],"""REAL_ESTATE""","""2742418167""",0,"[""COMMERCIAL""]","""OFFICE""","""ALL""",0,null,[],0,[],[],null,"[{""SALE"",1500,657300,0,{null,[],null}}]",[],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Centro Histórico""","""Rua Uruguai""","""155""","{""GOOGLE"",null,null,null,-30.028111,-51.22904}","""RS""","""STANDARD""",[175],null,"""dc0e9379-0d98-2ada-1128-b4e009…","""/imovel/venda-conjunto-comerci…"


A seguir, procuramos entender a coluna `pricingInfos`.

In [29]:
df.filter(pl.col("pricingInfos").list.len() > 1).head(3)

usableAreas,contractType,listingId,buildings,usageTypes,unitTypes,displayAddressType,unitFloor,suites,unitSubTypes,unitsOnTheFloor,bedrooms,amenities,floors,pricingInfos,parkingSpaces,listingType,bathrooms,resale,propertyType,city,neighborhood,street,streetNumber,point,stateAcronym,publicationType,totalAreas,capacityLimit,accountId,link_href
list[i64],str,str,i64,list[str],str,str,i64,i64,list[str],i64,list[i64],list[str],i64,list[struct[5]],list[i64],str,list[i64],bool,str,str,str,str,str,struct[6],str,str,list[i64],i64,str,str
[77],"""REAL_ESTATE""","""2577181447""",0,"[""COMMERCIAL""]","""OFFICE""","""NEIGHBORHOOD""",0,null,[],0,[],[],null,"[{""RENTAL"",106,1900,815,{null,[],null}}, {""SALE"",106,373867,815,{null,[],null}}]",[],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Centro Histórico""",null,null,"{""GOOGLE"",-30.028,-51.225,250,null,null}","""RS""","""STANDARD""",[77],null,"""6e401116-4d4c-1fd9-3d34-0fad4b…","""/imovel/venda-conjunto-comerci…"
[53],"""REAL_ESTATE""","""2722380679""",1,"[""RESIDENTIAL""]","""APARTMENT""","""ALL""",1,0,[],6,[1],"[""ELEVATOR"", ""CONCIERGE_24H"", … ""DISABLED_ACCESS""]",8,"[{""SALE"",60,180000,230,{null,[],null}}, {""RENTAL"",60,800,230,{null,[],null}}]",[0],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Centro Histórico""","""Rua Coronel Vicente""","""408""","{""GOOGLE"",null,null,null,-30.027116,-51.22086}","""RS""","""STANDARD""",[],null,"""0e1d53dc-d2a2-4386-8970-109201…","""/imovel/venda-apartamento-1-qu…"
[28],"""REAL_ESTATE""","""2758793231""",0,"[""RESIDENTIAL""]","""APARTMENT""","""ALL""",1,0,[],0,[1],[],null,"[{""RENTAL"",null,600,280,{null,[],null}}, {""SALE"",null,165000,280,{null,[],null}}]",[0],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Sarandi""","""Rua Gabriel Franco da Luz""","""205""","{""GOOGLE"",null,null,null,-29.985038,-51.123038}","""RS""","""STANDARD""",[28],null,"""ce00d3f1-8f07-386b-89a0-0563ad…","""/imovel/venda-apartamento-1-qu…"


In [30]:
df.select(pl.col("pricingInfos").explode().struct.field("businessType").unique())

businessType
str
"""RENTAL"""
"""SALE"""


Quando ocorre multiplicidade, é porque o imóvel pode ser comprado ou alugado.
Para nossos propósitos, nos interessamos tão somente em propriedades à venda, e apenas no valor de compra.
Desta forma, podemos descartar todas as outras informações e imóveis que não estão disponíveis para compra,
o que é revelado pelo atributo `businessType` de `pricingInfos`.

In [31]:
df = (
    df.explode(pl.col("pricingInfos"))
    .unnest("pricingInfos")
    .filter(pl.col("businessType") != "RENTAL")
)

In [32]:
df.height

15432

In [33]:
df.head(2)

usableAreas,contractType,listingId,buildings,usageTypes,unitTypes,displayAddressType,unitFloor,suites,unitSubTypes,unitsOnTheFloor,bedrooms,amenities,floors,businessType,yearlyIptu,price,monthlyCondoFee,rentalInfo,parkingSpaces,listingType,bathrooms,resale,propertyType,city,neighborhood,street,streetNumber,point,stateAcronym,publicationType,totalAreas,capacityLimit,accountId,link_href
list[i64],str,str,i64,list[str],str,str,i64,i64,list[str],i64,list[i64],list[str],i64,str,i64,i64,i64,struct[3],list[i64],str,list[i64],bool,str,str,str,str,str,struct[6],str,str,list[i64],i64,str,str
"[37, 38]","""REAL_ESTATE""","""2754214663""",0,"[""RESIDENTIAL""]","""APARTMENT""","""ALL""",0,null,[],0,[1],"[""INTERCOM"", ""KITCHEN""]",null,"""SALE""",1500,200000,300,"{null,[],null}",[0],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Cidade Baixa""","""Rua Luiz Afonso""","""558""","{""GOOGLE"",null,null,null,-30.041164,-51.223899}","""RS""","""PREMIUM""","[1, 43]",null,"""8c109097-7891-1cd2-f172-ac8f5c…","""/imovel/venda-apartamento-1-qu…"
[131],"""REAL_ESTATE""","""2742418167""",0,"[""COMMERCIAL""]","""OFFICE""","""ALL""",0,null,[],0,[],[],null,"""SALE""",1500,657300,0,"{null,[],null}",[],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Centro Histórico""","""Rua Uruguai""","""155""","{""GOOGLE"",null,null,null,-30.028111,-51.22904}","""RS""","""STANDARD""",[175],null,"""dc0e9379-0d98-2ada-1128-b4e009…","""/imovel/venda-conjunto-comerci…"


Podemos descartar `businessType`, que não é mais útil, e `rentalInfo`, pois não é relevante para nosso propósito.

In [34]:
df = df.drop("businessType", "rentalInfo")

A seguir, buscamos entender `unitSubTypes`.

In [35]:
df.filter(pl.col("unitSubTypes").list.len() > 1).head(3)

usableAreas,contractType,listingId,buildings,usageTypes,unitTypes,displayAddressType,unitFloor,suites,unitSubTypes,unitsOnTheFloor,bedrooms,amenities,floors,yearlyIptu,price,monthlyCondoFee,parkingSpaces,listingType,bathrooms,resale,propertyType,city,neighborhood,street,streetNumber,point,stateAcronym,publicationType,totalAreas,capacityLimit,accountId,link_href
list[i64],str,str,i64,list[str],str,str,i64,i64,list[str],i64,list[i64],list[str],i64,i64,i64,i64,list[i64],str,list[i64],bool,str,str,str,str,str,struct[6],str,str,list[i64],i64,str,str
[218],"""REAL_ESTATE""","""2755957813""",1,"[""RESIDENTIAL""]","""APARTMENT""","""ALL""",11,1,"[""PENTHOUSE"", ""DUPLEX""]",0,[3],"[""SAFETY_CIRCUIT"", ""BARBECUE_GRILL"", … ""CLOSET""]",11,2500,1187500,1500,[1],"""USED""",[3],false,"""UNIT""","""Porto Alegre""","""Cidade Baixa""","""Avenida Venâncio Aires""","""134""","{""GOOGLE"",null,null,null,-30.042264,-51.218778}","""RS""","""STANDARD""",[251],null,"""cee86b23-7ff3-d2f6-4227-0002e1…","""/imovel/venda-cobertura-3-quar…"
[165],"""REAL_ESTATE""","""2752739324""",0,"[""RESIDENTIAL""]","""APARTMENT""","""ALL""",0,1,"[""PENTHOUSE"", ""DUPLEX""]",4,[2],"[""AIR_CONDITIONING"", ""BATHROOM_CABINETS"", … ""GYM""]",8,3300,1070000,1668,[1],"""USED""","[2, 3]",false,"""UNIT""","""Porto Alegre""","""Menino Deus""","""Rua Gonçalves Dias""","""185""","{""GOOGLE"",null,null,null,-30.051703,-51.22068}","""RS""","""PREMIUM""","[203, 204]",null,"""7e682d41-74f9-029f-8684-cf6062…","""/imovel/venda-cobertura-2-quar…"
[112],"""REAL_ESTATE""","""2732984413""",0,"[""RESIDENTIAL""]","""APARTMENT""","""ALL""",0,null,"[""PENTHOUSE"", ""DUPLEX""]",0,[3],[],null,1170,439000,400,[1],"""USED""",[2],false,"""UNIT""","""Porto Alegre""","""Menino Deus""","""Rua Paes de Andrade""","""37""","{""GOOGLE"",null,null,null,-30.0585,-51.218554}","""RS""","""STANDARD""",[],null,"""d2e3b32c-6b0b-6173-45c8-6bb974…","""/imovel/venda-cobertura-3-quar…"


`unitSubTypes`, neste caso, define duas categorias que se aplicam ao imóvel.

Por outro lado, um imóvel pode não ter nenhum subtipo? E, se sim, isso é comum?

In [36]:
df.filter(pl.col("unitSubTypes").list.len() == 0).height

14743

De fato, esse é o mais comum dos casos. Se esse dado fosse presente para todos os registros,
poderia ser um atributo interessante. Como não é o caso, o descartamos.

In [37]:
df = df.drop("unitSubTypes")

In [38]:
df.head(3)

usableAreas,contractType,listingId,buildings,usageTypes,unitTypes,displayAddressType,unitFloor,suites,unitsOnTheFloor,bedrooms,amenities,floors,yearlyIptu,price,monthlyCondoFee,parkingSpaces,listingType,bathrooms,resale,propertyType,city,neighborhood,street,streetNumber,point,stateAcronym,publicationType,totalAreas,capacityLimit,accountId,link_href
list[i64],str,str,i64,list[str],str,str,i64,i64,i64,list[i64],list[str],i64,i64,i64,i64,list[i64],str,list[i64],bool,str,str,str,str,str,struct[6],str,str,list[i64],i64,str,str
"[37, 38]","""REAL_ESTATE""","""2754214663""",0,"[""RESIDENTIAL""]","""APARTMENT""","""ALL""",0,null,0,[1],"[""INTERCOM"", ""KITCHEN""]",null,1500,200000,300,[0],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Cidade Baixa""","""Rua Luiz Afonso""","""558""","{""GOOGLE"",null,null,null,-30.041164,-51.223899}","""RS""","""PREMIUM""","[1, 43]",null,"""8c109097-7891-1cd2-f172-ac8f5c…","""/imovel/venda-apartamento-1-qu…"
[131],"""REAL_ESTATE""","""2742418167""",0,"[""COMMERCIAL""]","""OFFICE""","""ALL""",0,null,0,[],[],null,1500,657300,0,[],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Centro Histórico""","""Rua Uruguai""","""155""","{""GOOGLE"",null,null,null,-30.028111,-51.22904}","""RS""","""STANDARD""",[175],null,"""dc0e9379-0d98-2ada-1128-b4e009…","""/imovel/venda-conjunto-comerci…"
[34],"""REAL_ESTATE""","""2596966563""",0,"[""RESIDENTIAL""]","""APARTMENT""","""ALL""",4,1,0,[1],"[""KITCHEN"", ""FURNISHED""]",4,350,135000,200,[],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Centro Histórico""","""Avenida João Pessoa""","""403""","{""GOOGLE"",null,null,null,-30.035665,-51.220879}","""RS""","""PREMIUM""",[41],null,"""8ce30dcf-d575-1b69-a46d-c41c78…","""/imovel/venda-apartamento-1-qu…"


In [39]:
df.filter(pl.col("bedrooms").list.len() > 1).head(5)

usableAreas,contractType,listingId,buildings,usageTypes,unitTypes,displayAddressType,unitFloor,suites,unitsOnTheFloor,bedrooms,amenities,floors,yearlyIptu,price,monthlyCondoFee,parkingSpaces,listingType,bathrooms,resale,propertyType,city,neighborhood,street,streetNumber,point,stateAcronym,publicationType,totalAreas,capacityLimit,accountId,link_href
list[i64],str,str,i64,list[str],str,str,i64,i64,i64,list[i64],list[str],i64,i64,i64,i64,list[i64],str,list[i64],bool,str,str,str,str,str,struct[6],str,str,list[i64],i64,str,str
[2500],"""REAL_ESTATE""","""2755316309""",0,"[""COMMERCIAL""]","""SHED_DEPOSIT_WAREHOUSE""","""ALL""",0,0,0,"[0, 1]","[""GARAGE"", ""PETS_ALLOWED"", ""KITCHEN""]",null,4000,10600000,0,"[20, 9]","""USED""","[1, 8]",false,"""UNIT""","""Porto Alegre""","""Sarandi""","""Avenida Plínio Kroeff""","""1200""","{""GOOGLE"",null,null,null,-30.001785,-51.116145}","""RS""","""STANDARD""","[2800, 7000]",null,"""ac071d10-5be9-069b-9b05-0e1fad…","""/imovel/venda-galpao-deposito-…"
"[17, 127]","""REAL_ESTATE""","""2752955525""",0,"[""RESIDENTIAL""]","""APARTMENT""","""ALL""",2,null,0,"[1, 2]","[""PETS_ALLOWED"", ""GATED_COMMUNITY"", … ""KITCHEN""]",4,800,152000,200,"[0, 25]","""USED""","[1, 2]",false,"""UNIT""","""Porto Alegre""","""Centro Histórico""","""Rua General Lima e Silva""","""407""","{""GOOGLE"",null,null,null,-30.03725,-51.223071}","""RS""","""PREMIERE_2""","[1, 691]",null,"""67c66b7c-4a43-e789-778d-ed4167…","""/imovel/venda-apartamento-2-qu…"
[176],"""REAL_ESTATE""","""2766308113""",0,"[""RESIDENTIAL""]","""PENTHOUSE""","""ALL""",0,1,0,"[2, 3]","[""INTERCOM"", ""SERVICE_AREA""]",8,83,800000,715,[],"""USED""","[1, 2]",false,"""UNIT""","""Porto Alegre""","""Centro Histórico""","""Rua Duque de Caxias""","""1515""","{""GOOGLE"",null,null,null,-30.033498,-51.226183}","""RS""","""STANDARD""","[176, 236]",null,"""7b72e174-9b60-84bf-44f7-79a9c2…","""/imovel/venda-cobertura-3-quar…"
[324],"""REAL_ESTATE""","""2744653296""",0,"[""RESIDENTIAL""]","""CONDOMINIUM""","""ALL""",0,1,0,"[3, 4]","[""BARBECUE_GRILL"", ""SERVICE_AREA"", ""CONCIERGE_24H""]",null,284,2100000,2100,[4],"""USED""","[3, 4]",false,"""UNIT""","""Porto Alegre""","""Menino Deus""","""Rua Silveiro""","""1007""","{""GOOGLE"",null,null,null,-30.069051,-51.231028}","""RS""","""STANDARD""","[324, 438]",null,"""7b72e174-9b60-84bf-44f7-79a9c2…","""/imovel/venda-casa-de-condomin…"
[96],"""REAL_ESTATE""","""2746482052""",1,"[""RESIDENTIAL""]","""APARTMENT""","""ALL""",2,1,0,"[2, 4]","[""KITCHEN_CABINETS"", ""BATHTUB"", … ""LAMINATED_FLOOR""]",3,1300,510000,440,[1],"""USED""","[1, 2]",false,"""UNIT""","""Porto Alegre""","""Menino Deus""","""Rua Barbedo""","""756""","{""GOOGLE"",null,null,null,-30.05755,-51.22319}","""RS""","""STANDARD""","[114, 10]",null,"""44b1655e-2eef-7133-303d-4a64c7…","""/imovel/venda-apartamento-4-qu…"


### Localização geográfica dos imóveis

Crucial para nossos esforços é ter a localização dos imóveis, para podermos cruzá-las
com o mapa de inundação.

In [40]:
df = df.unnest("point")
df.head(3)

usableAreas,contractType,listingId,buildings,usageTypes,unitTypes,displayAddressType,unitFloor,suites,unitsOnTheFloor,bedrooms,amenities,floors,yearlyIptu,price,monthlyCondoFee,parkingSpaces,listingType,bathrooms,resale,propertyType,city,neighborhood,street,streetNumber,source,approximateLat,approximateLon,radius,lat,lon,stateAcronym,publicationType,totalAreas,capacityLimit,accountId,link_href
list[i64],str,str,i64,list[str],str,str,i64,i64,i64,list[i64],list[str],i64,i64,i64,i64,list[i64],str,list[i64],bool,str,str,str,str,str,str,f64,f64,i64,f64,f64,str,str,list[i64],i64,str,str
"[37, 38]","""REAL_ESTATE""","""2754214663""",0,"[""RESIDENTIAL""]","""APARTMENT""","""ALL""",0,null,0,[1],"[""INTERCOM"", ""KITCHEN""]",null,1500,200000,300,[0],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Cidade Baixa""","""Rua Luiz Afonso""","""558""","""GOOGLE""",null,null,null,-30.041164,-51.223899,"""RS""","""PREMIUM""","[1, 43]",null,"""8c109097-7891-1cd2-f172-ac8f5c…","""/imovel/venda-apartamento-1-qu…"
[131],"""REAL_ESTATE""","""2742418167""",0,"[""COMMERCIAL""]","""OFFICE""","""ALL""",0,null,0,[],[],null,1500,657300,0,[],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Centro Histórico""","""Rua Uruguai""","""155""","""GOOGLE""",null,null,null,-30.028111,-51.22904,"""RS""","""STANDARD""",[175],null,"""dc0e9379-0d98-2ada-1128-b4e009…","""/imovel/venda-conjunto-comerci…"
[34],"""REAL_ESTATE""","""2596966563""",0,"[""RESIDENTIAL""]","""APARTMENT""","""ALL""",4,1,0,[1],"[""KITCHEN"", ""FURNISHED""]",4,350,135000,200,[],"""USED""",[1],false,"""UNIT""","""Porto Alegre""","""Centro Histórico""","""Avenida João Pessoa""","""403""","""GOOGLE""",null,null,null,-30.035665,-51.220879,"""RS""","""PREMIUM""",[41],null,"""8ce30dcf-d575-1b69-a46d-c41c78…","""/imovel/venda-apartamento-1-qu…"


Nem todas as listagens têm a localização precisa do imóvel. Algumas têm apenas uma
localização aproximada, dada por `approximateLat` e `approximateLon`.

In [41]:
df.filter(pl.col("approximateLat").is_null(), pl.col("lat").is_null())

usableAreas,contractType,listingId,buildings,usageTypes,unitTypes,displayAddressType,unitFloor,suites,unitsOnTheFloor,bedrooms,amenities,floors,yearlyIptu,price,monthlyCondoFee,parkingSpaces,listingType,bathrooms,resale,propertyType,city,neighborhood,street,streetNumber,source,approximateLat,approximateLon,radius,lat,lon,stateAcronym,publicationType,totalAreas,capacityLimit,accountId,link_href
list[i64],str,str,i64,list[str],str,str,i64,i64,i64,list[i64],list[str],i64,i64,i64,i64,list[i64],str,list[i64],bool,str,str,str,str,str,str,f64,f64,i64,f64,f64,str,str,list[i64],i64,str,str
[148],"""OWNER""","""2489898892""",0,"[""RESIDENTIAL""]","""HOME""","""ALL""",0,2,0,[3],[],null,900,620000,null,[3],"""USED""",[3],true,"""UNIT""","""Porto Alegre""","""Sarandi""","""Rua Doutor João Dahne""","""83""",null,null,null,null,null,null,"""RS""","""PREMIUM""",[],null,"""711f0095-bb5c-9ee7-de75-f73ec5…","""/imovel/venda-casa-3-quartos-s…"
[32],"""OWNER""","""2561009358""",0,"[""COMMERCIAL""]","""OFFICE""","""ALL""",14,null,0,[],[],null,null,280000,440,[1],"""USED""",[1],true,"""UNIT""","""Porto Alegre""","""Menino Deus""","""Avenida Getúlio Vargas""","""910""",null,null,null,null,null,null,"""RS""","""PREMIUM""",[],null,"""1847028f-f2f1-0d59-a208-01c230…","""/imovel/venda-conjunto-comerci…"


Duas listagens não constam nenhuma localização, de forma que devem ser descartadas.

In [42]:
df = df.filter(pl.col("approximateLat").is_not_null() | pl.col("lat").is_not_null())

Desta forma, nossa massa final de dados é:

In [43]:
df.height

15430

In [44]:
df.group_by("neighborhood").len().sort(by="len", descending=True)

neighborhood,len
str,u32
"""Centro Histórico""",6059
"""Menino Deus""",4814
"""Sarandi""",2550
"""Cidade Baixa""",2007


#### Cruzamento da localização dos imóveis com áreas alagadas

In [45]:
kml_data_file = "data/cheias_em_porto_alegre.kml"
polygons = extract_polygons_from_folder(
    kml_data_file, "Inundação em 6 de Maio de 2024"
)

In [46]:
df = mark_points_in_polygons(df, polygons)

In [54]:
tmp = (
    df.group_by("neighborhood")
    .agg(
        [
            pl.len().alias("total_count"),
            pl.col("flooded")
            .sum()
            .alias(
                "flooded_count"
            ),  # Since flooded is boolean, sum() gives us the count of True values
        ]
    )
    .with_columns(
        [
            (pl.col("flooded_count") / pl.col("total_count") * 100)
            .round(1)
            .alias("flooded_percentage")
        ]
    )
).to_pandas()
print(
    tabulate(
        tmp,
        headers=[
            "Bairro",
            "Número de Imóveis",
            "Número de Imóveis Alagados",
            "% Imóveis Alagados",
        ],
        tablefmt="latex",
        showindex=False
    )
)

\begin{tabular}{lrrr}
\hline
 Bairro           &   Número de Imóveis &   Número de Imóveis Alagados &   \% Imóveis Alagados \\
\hline
 Sarandi          &                2550 &                          534 &                 20.9 \\
 Centro Histórico &                6059 &                         1797 &                 29.7 \\
 Menino Deus      &                4814 &                         2582 &                 53.6 \\
 Cidade Baixa     &                2007 &                          835 &                 41.6 \\
\hline
\end{tabular}


In [372]:
df.filter(pl.col("flooded")).height

5748

In [373]:
df.filter(pl.col("flooded")).group_by("neighborhood").len().sort(
    by="len", descending=True
)

neighborhood,len
str,u32
"""Menino Deus""",2582
"""Centro Histórico""",1797
"""Cidade Baixa""",835
"""Sarandi""",534
